# DP1 target fields summary

The notebook computes statistics and summaries of all the data in the 7 DP1 fields. It computes a number of parameters to beuse din the paper<br>

Tables produced:<br> 
* Table 2: Summary of the DP1 fields.<br>
* Table X: tracts per fields for the datapriducts section <br>

Figures Produced: <br>
* Figure X: DP1 fields superimposed on survey baseline

## Install dependencies

Comment these out if already installed

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# Iinstall the scehduler data if not already done 
%run /opt/lsst/software/stack/conda/envs/lsst-scipipe-10.0.0/bin/scheduler_download_data

In [ ]:
# Generic python packages
import os
import csv
import requests
import yaml
import numpy as np
import pandas as pd
from pathlib import Path

# LSST Science Pipelines 
from lsst.daf.butler import Butler

# RSP
from lsst.rsp import get_tap_service

# Rubin scheduler
from rubin_scheduler.scheduler.utils import get_current_footprint
from rubin_scheduler.data.scheduler_download_data import scheduler_download_data
import healpy as hp
from rubin_nights.connections import get_access_token
from rubin_nights.consdb_query import ConsDbTap

# Set rubin publication plot style
from lsst.utils.plotting import publication_plots, get_multiband_plot_colors
import matplotlib.pyplot as plt
import colorcet as cc
from matplotlib.patches import Patch

## Setup

### Publication style

In [ ]:
# Setup publication style
publication_plots.set_rubin_plotstyle()
colors = get_multiband_plot_colors()
bands = colors.keys()  # important to get the right order for plot legends
bands_dict = publication_plots.get_band_dicts()
%matplotlib inline

### Utility functions

In [ ]:
# Custom formatter: no trailing zeros if not needed
def custom_float(x):
    if isinstance(x, float):
        s = f"{x:.3f}" #.rstrip('0').rstrip('.')
        return s
    return x

In [ ]:
# Function to round to N significant figures
def round_sf(x, sig=3):
    return np.round(x, sig - int(np.floor(np.log10(abs(x)))) - 1)

### Paths

In [ ]:
# Set up the figures, data and tables paths
figures_path = str(Path('../figures').resolve())
tables_path = str(Path('../tables').resolve())

# Set the scheduler data path
data_dir = Path('../scheduler_data')
os.environ['RUBIN_SIM_DATA_DIR'] = str(data_dir.resolve())
assert os.getenv('RUBIN_SIM_DATA_DIR') == str(data_dir.resolve())

# Where are we running
api_base = os.getenv("EXTERNAL_INSTANCE_URL")

### DP1 Butler

In [ ]:
instrument = 'LSSTComCam'
collections = ['LSSTComCam/DP1/defaults', 
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

### Consolidated DB over TAP

In [ ]:
# Rubin TAP service 
consdb = get_tap_service("consdbtap")
instrument = 'lsstcomcam'
assert consdb is not None

# Use the ConsDB client from rubin_nights to get extra information
# consdb_rn = ConsDbTap(api_base, 
#                    token = get_access_token())

# Extract the DP1 target fields 

In [ ]:
# Extract filter and pointing information about the fileds from the DP1 exposures in the Butler
exposures = registry.queryDimensionRecords('exposure')
exp_df = pd.DataFrame(columns=['id', 'target', 'physical_filter','ra', 'dec'])
for count, info in enumerate(exposures):
    try:
        exp_df.loc[count] = [info.id, info.target_name, info.physical_filter, 
                         info.tracking_ra, info.tracking_dec,]
    except: 
        print(">>>   Unexpected error:", sys.exc_info()[0])


In [ ]:
# Physical filter -> band
exp_df['band'] = exp_df['physical_filter'].str.split('_').str[0]

In [ ]:
exp_df

In [ ]:
## Summarise  the fields

In [ ]:
# slew_icrs is the ECDFS field -- combine them
exp_df.loc[exp_df['target'] == 'slew_icrs', 'target'] = 'ECDFS'

In [ ]:
# List of unique fields in DP1
dp1_fields = exp_df.target.unique()
assert len(dp1_fields) == 7

In [ ]:
# Number of images for each field, total and per band
n_images_all_bands = exp_df.groupby('target').size()
n_images_all_bands = n_images_all_bands.rename('Total').reset_index()

In [ ]:
# Compute the number of exposures per band and set the standard band order
n_images_per_band  = exp_df.groupby(['target', 'band']).size().unstack(fill_value=0)[bands]
assert n_images_per_band.to_numpy().sum() == exposures.count()

In [ ]:
n_images_summary = pd.merge(n_images_per_band, n_images_all_bands, on='target', how='left')
n_images_summary

In [ ]:
# Manually insert a description column
n_images_summary.insert(1, 'Field Name',[
    "47 Tucanae Globular Cluster",
    "Extended Chandra Deep Field South",
    "Rubin SV Euclid Deep Field South",
    "Fornax Dwarf Spheroidal Galaxy",
    "Rubin SV Low Galactic Latitude Field",
    "Rubin SV Low Ecliptic Latitude Field",
    "Seagull Nebula"])
n_images_summary

In [ ]:
# Compute the median of all pointings per field for the 
# pointing centers and add to the summary table
target_centers = exp_df.groupby('target')[['ra', 'dec']].agg(['median'])
target_centers.columns = ['_'.join(col) for col in target_centers.columns]
target_centers
target_centers = target_centers.map(lambda x: round_sf(x, 4))
dp1_targets_summary = pd.merge(n_images_summary, target_centers, on="target")

In [ ]:
dp1_targets_summary

In [ ]:
# Rename and reorder 
dp1_targets_summary.rename(columns={
    'target': "Field Code",
    'ra_median': 'RA',
    'dec_median': 'DEC'}, inplace=True)

# Reorder columns by specifying the new order
dp1_targets_summary.insert(2, 'RA', dp1_targets_summary.pop('RA'))
dp1_targets_summary.insert(3, 'DEC', dp1_targets_summary.pop('DEC'))


In [ ]:
# Format the date to produce a latex table
# Insert blank colum for nice spacing in table 
dp1_targets_summary.insert(4, ' ', ' ')

# Escape the underscores in the field names and codes
dp1_targets_summary.columns = dp1_targets_summary.columns.str.replace('_', r'\_', regex=False)
dp1_targets_summary = dp1_targets_summary.map(lambda x: x.replace('_', r'\_') if isinstance(x, str) else x)

dp1_targets_summary_latex = dp1_targets_summary.to_latex(index=False, 
                                                         escape=False, 
                                                         bold_rows=False,
                            formatters={col: custom_float 
                                        for col in dp1_targets_summary.columns}
                            )

# Remove unnecessary latex
data_latex = dp1_targets_summary_latex.split("midrule\n", 1)[-1]  # Keeps the part after 'midrule'
data_latex = data_latex.split("\\bottomrule", 1)[0]  # Keeps the part before 'bottomrule'

In [ ]:
dp1_targets_summary

In [ ]:
# Export to latex with deulxetable formatting 
with open("../tables/dp1_fields_summary.tex", "w") as f:
    f.write(r"""%%%%% This table is auto generated from data, DO NOT EDIT
\begin{deluxetable}{llcccp{0.5cm}p{0.6cm}p{0.3cm}p{0.3cm}p{0.3cm}p{0.3cm}r}
\caption{DP1 fields and pointing centers with the number of images in each band per field.  
ICRS coordinates are in units of decimal degrees. 
\label{tab:dp1_fields} }
\tablehead{
  \colhead{\textbf{Field Code}} & \colhead{\textbf{Field Name}} & \colhead{\textbf{RA}} & \colhead{\textbf{DEC}} 
  & & \multicolumn{6}{c}{\textbf{Band}} & \colhead{\textbf{Total}}\\
  \cline{3-4} \cline{6-11} 
  & & \colhead{deg}  & \colhead{deg}  & & u & g & r & i & z & y & 
}
\startdata
""")
    f.write(data_latex)
    f.write(r"""\enddata
\end{deluxetable}
""")
f.close()

## Figure showing DP1 fields with baseline survey areas in background

In [ ]:
footprint_hp, labels = get_current_footprint(nside=64)
np.unique(labels)

In [ ]:
# Refactor this into the rubin plotstyle
region_dict = {'scp': 0, 'nes': 3, 'dusty_plane': 4, 'gpwfd': 6, 'lowdust': 7} # better for viridis

In [ ]:
max(region_dict.values())

In [ ]:
# Plot 
total_fp = np.zeros(len(labels))
cmap = 'viridis'
norm = plt.Normalize(vmin=0, vmax=max(region_dict.values()))

total_fp = np.where(labels == 'scp', region_dict['scp'], total_fp)
total_fp = np.where(labels == 'nes', region_dict['nes'], total_fp)
total_fp = np.where((labels == 'dusty_plane'), region_dict['dusty_plane'], total_fp)
total_fp = np.where(((labels == 'bulgy') | (labels == 'LMC_SMC')), region_dict['gpwfd'], total_fp)
total_fp = np.where(((labels == 'euclid_overlap') | (labels == 'lowdust') | (labels == 'virgo')), region_dict['lowdust'], total_fp)
total_fp = np.where(labels == '', np.nan, total_fp)

norm = plt.Normalize(vmin=0, vmax=7)
hp.mollview(total_fp, badcolor='white', cmap=cmap, norm=norm, cbar=False, title=None)
hp.graticule(alpha=0.5)

ax = plt.gca()
im = ax.get_images()[0]

legend_elements = [Patch(facecolor=im.get_cmap()(norm(region_dict['scp'])), label="SCP"),
                   Patch(facecolor=im.get_cmap()(norm(region_dict['nes'])), label="NES"),
                   Patch(facecolor=im.get_cmap()(norm(region_dict['dusty_plane'])), label="Dusty Plane"),
                   Patch(facecolor=im.get_cmap()(norm(region_dict['gpwfd'])), label="GP/MC WFD"),
                   Patch(facecolor=im.get_cmap()(norm(region_dict['lowdust'])), label="LowDust WFD")]
ax.legend(handles=legend_elements, loc=(0.14, -0.1), ncols=5)

for counter, (i, target) in enumerate(dp1_targets_summary.iterrows()):
    name = target['Field Code'].replace("\\", '')
    sign = 2 * (counter % 2) + 1
    hp.projscatter(target.RA, target.DEC, lonlat=True, facecolor='black', 
                   marker='o', edgecolors='white' )
    if name.startswith('Rubin'):
        hp.projtext(target['RA'] + 20, target['DEC'] + 5, name,  lonlat=True)
    elif name.startswith('ECDFS'):
        hp.projtext(target['RA'] + 38, target['DEC'] -5 , name, lonlat=True)
    else:
        hp.projtext(target['RA'] + 3, target['DEC'] + 5, name, lonlat=True)

plt_path = os.path.join(figures_path, "dp1_fields_with_survey_fp.pdf")
plt.savefig(plt_path, format='pdf', bbox_inches='tight')

In [ ]:
plt.close